In [ ]:
%load_ext autoreload
%autoreload 2

from algorithms.envs.NCS.cacc_env import CACCEnv
import configparser

input: action

output: state reward, done, global_reward

8 vehicles

([v_state, vdiff_state, vhdiff_state, h_state, u_state])

v-v_star/v_star, v difference from car', vhdiff, distance from car', acceleration

code for vhdiff:
        vh = self.ovm.get_vh(self.hs_cur[i_veh])
        vhdiff_state = np.clip((vh - self.vs_cur[i_veh]) / VDIFF, -2, 2)
basically it tells the agent what vh is. 

At first all cars dirve at optiaml v, The first car is guided by its headway... 

 here we train NMARL to recommend appropriate (α◦
, β◦) for
each OVM controller, selected from four levels {(0, 0),(0.5, 0),(0, 0.5),(0.5, 0.5)}

For both CACC tasks, we simulate a string of 8 vehicles for 60s, with a 0.1s
control interval. Each vehicle observes and shares its headway h, velocity v, and acceleration a to
neighbors within two steps. The safety constraints are: h ≥ 1m, v ≤ 30m/s, |a| ≤ 2.5m/s2
.
Both v and h are normalized

alpha: weight of headway
beta: weight of target speed (for all following cars, set as the speed of the car before)
for the first car, it is always v_star in the code

runs for 600 frames, 60s sec * 10 per sec

# Initialize Env from Scratch

In [ ]:
import numpy as np
output_path = '/tmp/'
config_path = 'algorithms/envs/CACC/config/config_ma2c_nc_catchup.ini'
config = configparser.ConfigParser()
config.read(config_path)

In [ ]:

env = CACCEnv(config['ENV_CONFIG'])
env.init_data(True, False, output_path)
ob = env.reset()
while True:
    ob, _, done, _ = env.step(np.array([2]*(env.n_agent)))
    if done:
        break
result = env.output_data()


# Use Env Config and Wrapper

In [49]:
from algorithms.envs.CACC import env_name, env_fn

> /home/asus/RL-algorithms/algorithms/envs/CACC.py(33)_k_hop()
     31     def _k_hop(self, state):
     32         pdb.set_trace()
---> 33         result = np.zeros((8, 5*(1+2*self.k)), dtype = np.float32)
     34         for i in range(8):
     35             for j in range(i-self.k, i+self.k+1):



ipdb>  n


> /home/asus/RL-algorithms/algorithms/envs/CACC.py(34)_k_hop()
     32         pdb.set_trace()
     33         result = np.zeros((8, 5*(1+2*self.k)), dtype = np.float32)
---> 34         for i in range(8):
     35             for j in range(i-self.k, i+self.k+1):
     36                 if j<0 or j>=8:



ipdb>  result.shape


(8, 15)


ipdb>  exit


[autoreload of algorithms.envs.CACC failed: Traceback (most recent call last):
  File "/home/asus/anaconda3/envs/MARL/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/home/asus/anaconda3/envs/MARL/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/home/asus/anaconda3/envs/MARL/lib/python3.7/imp.py", line 315, in reload
    return importlib.reload(module)
  File "/home/asus/anaconda3/envs/MARL/lib/python3.7/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 630, in _exec
  File "<frozen importlib._bootstrap_external>", line 728, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "/home/asus/RL-algorithms/algorithms/envs/CACC.py", line 61, in <module>
    result  = np.array(env.reset())
  File "/home/asus/RL-

In [ ]:
env = env_fn()
env.reset()
state, reward, done, info = env.step([1]*8)

In [41]:
from gym.spaces import Box

# Visualization

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

from matplotlib import animation, rc
from IPython.display import HTML
# First set up the figure, the axis, and the plot element we want to animate
fig, ax = plt.subplots()

ax.set_xlim(( -2, 2))
ax.set_ylim((-2, 2))

scatter = ax.scatter([0, 1], [0, 0])
ob = env.reset()

In [ ]:
h_star = 20


# initialization function: plot the background of each frame
def init():
    ob = env.reset()
    xs = []
    ys = [0]*8
    tmp = 0
    for i, item in enumerate(ob):
        tmp -= item[-2]*h_star + h_star
        xs += [tmp]
    colors = ['red', 'orange', 'yellow', 'green', 'cyan', 'blue', 'purple', 'black']
    scatter = ax.scatter(xs, ys, c=colors)
    return (scatter,)

# animation function. This is called sequentially

def animate(i):
    ob, _, done, _ = env.step([3]*(env.n_agent))
    if done:
        raise
    tmp = 0
    xs = []
    ys = [0]*8
    for i, item in enumerate(ob):
        tmp -= item[-2]* h_star + h_star
        xs += [tmp]
    ax.clear()
    ax.set_xlim(( -190, 0))
    ax.set_ylim((-2, 2))
    colors = ['red', 'orange', 'yellow', 'green', 'cyan', 'blue', 'purple', 'black']
    scatter = ax.scatter(xs, ys, c=colors)
    return (scatter,)

# call the animator. blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=599, interval=100, blit=True)
HTML(anim.to_html5_video())

In [ ]:
anim.save("tmp.mp4")

In [ ]:
env.ifCollide(ob)